In [1]:
%matplotlib inline 

import os
import glob
import numpy as np
import pandas as pd
from skimage import io 
import rasterio
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:

# mu_sd= np.array([])
# sca_km= np.array([])
# p_basin = np.array([])

sd_lst = np.array([])
sc_lst = np.array([])
pb_lst = np.array([])

In [3]:
# path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/'
path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test'
#path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/20170401_clipped.tif'


##LOOP - change path to f....2 spots)
for f in sorted(glob.glob(os.path.join(path, '*.tif'))):
    print(f)
    raster = io.imread(f) #path
    # print("raster shape",raster.shape)

    # io.imshow(raster[:,:,0]) #snow depth, band1
    # io.show()

    # io.imshow(raster[:,:,1]) #alpha mask, band2
    # io.show()


    # convert to numpy array
    array = np.array(raster[:,:,0]) #RAW array of lidar tif
    # print("array shape",array.shape)

    #Ernesto method....
    # a=array.copy()
    sno_mask = array.copy()
    
    sno_mask[sno_mask > 0] = 1 #make SCA mask

    #sno_mask is matrix of 0 (no snow) and 1's (snow)
#     plt.imshow(sno_mask,alpha=None)
#     plt.title('Mask Layer (1=snow cover)')
#     plt.show()

    # meanSD -- caclulate mean snow depth (meanSD), without zero snow pixels
    mu_sd = np.nanmean(sno_mask) #mean snow depth, from sca (SOME LIDAR HAVE NANS IN THE CLIPPED)
    
#     print("mean snow depth", mu_sd,'meters')


    # SCA -- pixs snow cover * pixel size
    num_pix = np.nansum(sno_mask) #number of pixels with snow cover (sum the 1's)
#     print("number pixels with snow cover",num_pix)
    
    ras =  rasterio.open(f) #CHANGE FOR LOOP
    gt = ras.transform
    pixelSizeX = gt[0]
    pixelSizeY =-gt[4] #notice the -gt
#     print('pixel size (x,y) = ',pixelSizeX, pixelSizeY)

    sca = num_pix * pixelSizeX * pixelSizeY
    sca_km = sca/1000**2
    p_basin = sca_km/1180 *100  #per Hedrick et al. 2018
#     print('Snow Covered Area (SCA) = ', sca, 'm^2')
#     print('Snow Covered Area (SCA) = ', sca_km, 'km^2')
#     print('% Basin snow cover', p_basin, '%')
    
#     array.close()
#     sno_mask.close()
#     ras.close()
    sd_lst = np.append(sd_lst,mu_sd)
    sc_lst = np.append(sc_lst,sca_km)
    pb_lst = np.append(pb_lst,p_basin)


df1 = pd.DataFrame(zip(sd_lst,sc_lst,pb_lst),columns = ['meanSD (m)','SCA (km)','% basin'])
df1

/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170129_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170303_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170401_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170502_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170604_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170709_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170717_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/batch_test/20170727_clipped.tif
/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipp

,meanSD (m),SCA (km),% basin
0,0.376816,9.997786e+02,8.472700e+01
1,0.384862,1.021126e+03,8.653614e+01
2,0.375572,9.964775e+02,8.444724e+01
3,0.352039,9.340390e+02,7.915585e+01
4,0.282234,7.488316e+02,6.346030e+01
5,0.089248,2.367961e+02,2.006746e+01
6,0.055746,1.479055e+02,1.253436e+01
7,-5726.493164,-1.519368e+07,-1.287600e+06
8,0.009481,2.515599e+01,2.131864e+00
9,-5545.472168,-1.471339e+07,-1.246898e+06


In [ ]:
fig, a=plt.subplots(3,1)

a[0].plot(df1['meanSD (m)'])
a[1].plot(df1['SCA (km)'])
a[2].plot(df1['% basin'])

# sns.set_style('darkgrid')
# sns.set_style("ticks")
# sns.set_context('notebook')

In [ ]:
# mu_sd= []
# sca_km= []
# p_basin = []

foo = [1,2,3]
moo = [4,5,6]
boo = [7,8,9]

col=['f','m','b']
 
# big_list = [(foo,moo,boo)]
big_list = [(foo,moo,boo)]
# print(big_list)

# df1 = pd.DataFrame(big_list,columns=col)
df1 = pd.DataFrame(zip(foo,moo,boo),columns=col)
df1

